# **Group Members:**
- Mohammad Anas (20L-1289)
- Fatima Azfar (20L-1027)
- Section: BDS-8A

In [16]:
# !pip install -qqq simpletransformers

In [ ]:
import pandas as pd
from simpletransformers.question_answering import QuestionAnsweringModel
from sklearn.model_selection import train_test_split
import wandb

In [ ]:
data = pd.read_csv("train.csv")
train_df, eval_df = train_test_split(data, test_size=0.25, random_state=42)

In [ ]:
model_configs = {
    "bert": {
        "model_type": "bert",
        "model_name": "bert-base-uncased",
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "num_train_epochs": 3,
        "learning_rate": 5e-5,
        "wandb_project": "MedQuad_QA_BERT"
    },
    "mobilebert": {
        "model_type": "bert",
        "model_name": "google/mobilebert-uncased",
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "num_train_epochs": 3,
        "learning_rate": 5e-5,
        "wandb_project": "MedQuad_QA_MobileBERT"
    },
    "roberta": {
        "model_type": "roberta",
        "model_name": "roberta-base",
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "num_train_epochs": 3,
        "learning_rate": 5e-5,
        "wandb_project": "MedQuad_QA_RoBERTa"
    }
}

hyperparameters = [
    {"num_train_epochs": 3, "learning_rate": 5e-5, "dropout": 0.3},
    {"num_train_epochs": 3, "learning_rate": 5e-5, "dropout": 0.7}
]


In [ ]:
def preprocess_data(df):
    examples = []
    for idx, row in df.iterrows():
        context = row["Answer"]
        qas = [{"question": row["Question"], "id": idx, "answers": [{"text": row["Answer"], "answer_start": 0}]}]
        examples.append({"context": context, "qas": qas})
    return examples

for model_name, config in model_configs.items():
    for hp in hyperparameters:
        # Update hyperparameters
        config.update(hp)

        train_examples = preprocess_data(train_df)
        eval_examples = preprocess_data(eval_df)

        wandb.init(project=config["wandb_project"], config=config)

        model = QuestionAnsweringModel(model_type=config["model_type"],
                                       model_name=config["model_name"],
                                       args=config)
        output_dir = f"{model_name} {hp}"
        model.train_model(train_examples, output_dir=output_dir)  # Add overwrite_output_dir=True

        result = model.eval_model(eval_examples)
        wandb.log({"eval_results": result})


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
add example index and unique id: 100%|██████████| 12305/12305 [00:00<00:00, 582364.74it/s]


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/974 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Running Epoch 2 of 3:   0%|          | 0/974 [00:00<?, ?it/s]

Running Epoch 3 of 3:   0%|          | 0/974 [00:00<?, ?it/s]

add example index and unique id: 100%|██████████| 4102/4102 [00:00<00:00, 311048.67it/s]


Running Evaluation:   0%|          | 0/320 [00:00<?, ?it/s]

Training loss,█▃▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▃▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr,▃▅▇███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
Training loss,0.00127
global_step,2900
lr,0.0


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
convert squad examples to features:   0%|          | 0/12305 [00:00<?, ?it/s]